# Запрос информации о количестве посещений страниц поэтов в русскоязычной Википедии

Анализируем число заходов на страницу по месяцам.

In [3]:
import pandas as pd

poets = pd.read_csv('data/poets-joined.csv')
poets.rename(columns={'Unnamed: 0': 'title'}, inplace=True)

poets

,title,pageid,ns,por_pageid,por_ns,rwp_pageid,rwp_ns
0,Проект:Литература/Списки/Русские поэты Украины,89076.0,104.0,NaN,NaN,NaN,NaN
1,Проект:Литература/Списки/Русские поэты XIX века,87816.0,104.0,NaN,NaN,NaN,NaN
2,Поэтессы Серебряного века,4692805.0,0.0,NaN,NaN,NaN,NaN
3,Русские поэты Серебряного века,87820.0,0.0,NaN,NaN,NaN,NaN
4,Русские поэты-футуристы,89092.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
3529,Яжевика,NaN,NaN,NaN,NaN,3331686.0,0.0
3530,"Якушева, Ада",NaN,NaN,NaN,NaN,79145.0,0.0
3531,"Янбулатова, Рагида Саитгалеевна",NaN,NaN,NaN,NaN,3788328.0,0.0
3532,Категория:Поэтессы Российской империи,NaN,NaN,NaN,NaN,3412574.0,14.0


## Запрашиваем статистику для страниц

Описание API [здесь](https://wikitech.wikimedia.org/wiki/Analytics/AQS/Pageviews).

Данные доступны начиная в 01.08.2015.

In [4]:
import requests

S = requests.Session()

start_date = '20150801'
end_date = '20201130'

def request_stats(article, start_date, end_date):
    URL = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/ru.wikipedia/all-access/all-agents/{0}/monthly/{1}/{2}'\
        .format(article, start_date, end_date)

    r = S.get(URL)
    data = r.json()

    if 'items' not in data:
        return pd.Series()

    return pd.Series({item['timestamp']:item['views'] for item in data['items']})

result = poets.merge(poets.apply(lambda x: request_stats(x['title'], start_date, end_date), axis=1), left_index=True, right_index=True)
result

<ipython-input-4-8727af38e5e4>:16: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()


,title,pageid,ns,por_pageid,por_ns,rwp_pageid,rwp_ns,2015080100,2015090100,2015100100,...,2020020100,2020030100,2020040100,2020050100,2020060100,2020070100,2020080100,2020090100,2020100100,2020110100
0,Проект:Литература/Списки/Русские поэты Украины,89076.0,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Проект:Литература/Списки/Русские поэты XIX века,87816.0,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Поэтессы Серебряного века,4692805.0,0.0,NaN,NaN,NaN,NaN,1349.0,1841.0,2456.0,...,2944.0,2874.0,3249.0,3053.0,2294.0,2103.0,2294.0,2242.0,2545.0,2475.0
3,Русские поэты Серебряного века,87820.0,0.0,NaN,NaN,NaN,NaN,5389.0,14159.0,22557.0,...,9191.0,10400.0,16523.0,15185.0,9723.0,3921.0,5854.0,9789.0,10338.0,11500.0
4,Русские поэты-футуристы,89092.0,0.0,NaN,NaN,NaN,NaN,831.0,6138.0,8135.0,...,4200.0,4400.0,9639.0,7992.0,6620.0,1538.0,1101.0,4509.0,5848.0,5218.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3529,Яжевика,NaN,NaN,NaN,NaN,3331686.0,0.0,1512.0,1528.0,1398.0,...,400.0,384.0,674.0,467.0,413.0,414.0,370.0,318.0,365.0,378.0
3530,"Якушева, Ада",NaN,NaN,NaN,NaN,79145.0,0.0,1222.0,7037.0,4053.0,...,2744.0,2427.0,5053.0,3766.0,3432.0,2634.0,3629.0,4404.0,4972.0,4209.0
3531,"Янбулатова, Рагида Саитгалеевна",NaN,NaN,NaN,NaN,3788328.0,0.0,73.0,77.0,126.0,...,96.0,88.0,98.0,152.0,63.0,89.0,79.0,57.0,70.0,52.0
3532,Категория:Поэтессы Российской империи,NaN,NaN,NaN,NaN,3412574.0,14.0,106.0,119.0,121.0,...,59.0,70.0,77.0,67.0,49.0,53.0,52.0,61.0,42.0,67.0


In [5]:
result.to_csv('data/poet-page-visit-stats.csv', encoding='utf8')